In [2]:
from ngsolve import *
from netgen.csg import *
from ngsolve.webgui import Draw

# ------------------------------------------------
# 1. 形状定義とメッシュ生成 (Geometry & Mesh)
# ------------------------------------------------
# CSG (Constructive Solid Geometry) ジオメトリを作成
geo = CSGeometry()

# (0,0,0) から (1,1,1) までの直方体（単位立方体）を追加
geo.Add(OrthoBrick(Pnt(0,0,0), Pnt(1,1,1)))

# ジオメトリからメッシュを生成 (maxhはメッシュの粗さを指定)
mesh = Mesh(geo.GenerateMesh(maxh=0.3))
# 解説: maxh=0.3 は、要素の最大サイズを0.3に設定しています。

# ------------------------------------------------
# 2. 有限要素空間の定義 (Finite Element Space)
# ------------------------------------------------
# H1空間（連続な関数空間）を定義。order=2 は2次の要素を使用することを意味します。
fes = H1(mesh, order=2)

# 試験関数 (Trial function) u と テスト関数 (Test function) v を定義
u, v = fes.TnT()

# ------------------------------------------------
# 3. 弱形式の定義 (Weak Form - Poisson Equation)
# ------------------------------------------------
# 左辺（双線形形式）の定義: スチフネス行列 A に相当
# a(u, v) = ∫ ∇u・∇v dx
a = BilinearForm(fes)
a += grad(u) * grad(v) * dx
a.Assemble() # 行列の組み立て

# 右辺（線形形式）の定義: 荷重ベクトル f に相当
# f(v) = ∫ 1 * v dx (ソース項は定数 1)
f = LinearForm(fes)
f += 1 * v * dx
f.Assemble() # ベクトルの組み立て

# ------------------------------------------------
# 4. ソルバー (Solve)
# ------------------------------------------------
# 解を格納するためのグリッド関数を作成
gfu = GridFunction(fes)

# 連立一次方程式 Ax = f を解く (Aの逆行列 * f)
gfu.vec.data = a.mat.Inverse() * f.vec

# ------------------------------------------------
# 5. 可視化 (Visualize)
# ------------------------------------------------
# ウェブベースのGUIで解とメッシュを描画
Draw(gfu, mesh, "solution")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [ ]:
from ngsolve import *
from netgen.csg import *
from ngsolve.webgui import Draw
from ngsolve.solvers import CG # 反復法ソルバーをインポート
import math

# ------------------------------------------------
# 1. 形状とメッシュ (Geometry & Mesh)
# ------------------------------------------------
geo = CSGeometry()

R_coil = 1.0
r_wire = 0.2

# コイル形状 (引数は数値のみ)
coil = Torus(Pnt(0,0,0), Vec(0,0,1), R_coil, r_wire)
coil.mat("coil")

# 空気領域 (引数は数値のみ)
air = Sphere(Pnt(0,0,0), 3.0)
air.mat("air")
air.bc("outer") # 境界条件名を設定

geo.Add(air - coil)
geo.Add(coil)

# メッシュ生成
# 反復法を使うので、少し細かめ(maxh=0.8)でもメモリ不足になりにくいです
mesh = Mesh(geo.GenerateMesh(maxh=0.8))
mesh.Curve(3)

# ------------------------------------------------
# 2. 物理定数とパラメータ
# ------------------------------------------------
mu0 = 4 * math.pi * 1e-7
mur = 1.0
nu = 1 / (mu0 * mur)

current_I = 1.0
area = math.pi * r_wire**2
J_mag = current_I / area
dir_vec = CF( (-y, x, 0) )
J = J_mag * dir_vec / Norm(dir_vec)

# ------------------------------------------------
# 3. 有限要素空間
# ------------------------------------------------
# 精度を確保するため order=2 を維持
fes = HCurl(mesh, order=2, dirichlet="outer")
u, v = fes.TnT()

# ------------------------------------------------
# 4. 弱形式と前処理 (Weak Form & Preconditioner)
# ------------------------------------------------
a = BilinearForm(fes)
a += nu * curl(u) * curl(v) * dx
a += 1e-6 * nu * u * v * dx # 正則化項

# 【重要変更1】前処理(Preconditioner)の定義
# 反復法の収束を助けるための設定です。"local" (Jacobi) はメモリ消費が最小です。
c = Preconditioner(a, "local") 

a.Assemble()

f = LinearForm(fes)
f += J * v * dx("coil")
f.Assemble()

# ------------------------------------------------
# 5. ソルバー (Iterative Solver: CG)
# ------------------------------------------------
gfu = GridFunction(fes)

# 【重要変更2】Inverse() の代わりに CGソルバーを使用
# tol: 許容誤差, maxsteps: 最大反復回数
print("Solving linear system with CG method...")
CG(mat=a.mat, pre=c.mat, rhs=f.vec, sol=gfu.vec, tol=1e-8, maxsteps=2000, printrates=True)

# ------------------------------------------------
# 6. インダクタンスの算出
# ------------------------------------------------
B = curl(gfu)
energy = 0.5 * Integrate(nu * B * B, mesh)
L = 2 * energy / (current_I**2)

print(f"--------------------------------")
print(f"Magnetic Energy: {energy:.6e} [J]")
print(f"Inductance:      {L:.6e} [H]")
print(f"--------------------------------")

# ------------------------------------------------
# 7. 可視化
# ------------------------------------------------
Draw(Norm(B), mesh, "B_Field", clipping={"z":0, "dist":0})

Solving linear system with CG method...
CG iteration 1, residual = 4.91132912552057e-05     
CG iteration 2, residual = 9.208793335161128e-05     
CG iteration 3, residual = 0.00010387170512776172     
CG iteration 4, residual = 0.00012692218806299244     
CG iteration 5, residual = 0.000152226677092142     
CG iteration 6, residual = 0.00018123003026880943     
CG iteration 7, residual = 0.00020606673728860872     
CG iteration 8, residual = 0.0002231725905226265     
CG iteration 9, residual = 0.00023658423600369055     
CG iteration 10, residual = 0.00024366022389130795     
CG iteration 11, residual = 0.00023988789919452043     
CG iteration 12, residual = 0.00022840784412970358     
CG iteration 13, residual = 0.0002139842779751811     
CG iteration 14, residual = 0.00020006757923853833     
CG iteration 15, residual = 0.0001880373572911356     
CG iteration 16, residual = 0.00017804584377051764     
CG iteration 17, residual = 0.00017133485868924564     
CG iteration 18, residual